In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [3]:
city_list = city_link_collector()
len(city_list)

415

In [4]:
def craigslist_motorcycle_scraper(city_list):
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts']
    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        city_response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(city_response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            
            #get request      
            post_response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if post_response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))

            #define the html text
            post_soup = BeautifulSoup(post_response.text, 'html.parser')

            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                count += 1
                
#               checks to see if post html is loaded in our database and scrapes html if not
                if not (post_html.find_one({'_id': link}, {'html': requests.get(link).text})):
                    print(str(link))
                    sub_post = requests.get(link)
                    post_html.insert_one({'_id': link, 'html': sub_post.text})
                    if sub_post.status_code != 200:
                        warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))
                else:
                    print('Passing ' + str(link))
                    pass
                time.sleep(random.randint(2,3)) #sleep timer to avoid being banned               
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            

        print("\n")
        print("{} complete!".format(city.title()))
        print('~' + str(len(pages)*120) + " rows collected.")
        print("\n")

In [5]:
client = MongoClient('localhost', 27017)
db = client['craigslist_motorcycles']
post_html = db['motorcycle_posts']

In [ ]:
craigslist_motorcycle_scraper(city_list)

https://auburn.craigslist.org/ Posts = 7
Https://Auburn.Craigslist.Org/ Pages = 1
https://auburn.craigslist.org/mcy/d/auburn-university-2016-honda-cbr300r/7120429265.html
https://auburn.craigslist.org/mcd/d/fayetteville-bad-credit-no-credit-ok-we/7119776172.html
https://auburn.craigslist.org/mcy/d/montgomery-2017-yamaha-fz-07-very-low/7118845773.html
https://auburn.craigslist.org/mcd/d/haleyville-2010-yellow-goldwing-gl1800/7116263343.html
https://auburn.craigslist.org/mcy/d/auburn-2006-honda-xr650l-price-drop/7114333691.html
https://auburn.craigslist.org/mcy/d/auburn-2016-harley-davidson-roadster/7114312352.html
https://auburn.craigslist.org/mcy/d/opelika-2017-concours-1400/7111191074.html
Https://Auburn.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Auburn.Craigslist.Org/ complete!
~120 rows collected.


https://bham.craigslist.org/ Posts = 104
Https://Bham.Craigslist.Org/ Pages = 1
https://bham.craigslist.org/mcy/d/tuscaloosa-2008-harley-davidson/7122367787.html
h

https://bham.craigslist.org/mcy/d/birmingham-harley-davidson-2008-ultra/7110896840.html
https://bham.craigslist.org/mcy/d/birmingham-2008-suzukki-birmingham/7110846047.html
https://bham.craigslist.org/mcy/d/cullman-2015-harley-davidson-heritage/7110489021.html
https://bham.craigslist.org/mcd/d/athens-2020-harley-davidson-sport-glide/7110379121.html
https://huntsville.craigslist.org/mcd/d/athens-2020-harley-davidson-sport-glide/7110376064.html
https://bham.craigslist.org/mcy/d/trussville-harley-davidson-heritage/7109819844.html
https://bham.craigslist.org/mcd/d/bessemer-2011-harley-davidson-street/7109689708.html
https://bham.craigslist.org/mcd/d/bessemer-2014-yamaha-xvs95ce/7109689512.html
https://bham.craigslist.org/mcd/d/hattiesburg-2019-yamahar3like-new-rare/7109634776.html
https://bham.craigslist.org/mcd/d/hattiesburg-2019-ssrsr70-automatic/7109583320.html
https://bham.craigslist.org/mcy/d/pelham-nice-scooter-cheap/7109429917.html
https://bham.craigslist.org/mcy/d/trussville-gold-w

https://huntsville.craigslist.org/mcd/d/florence-2010-kawasaki-concours-14-abs/7120242803.html
https://huntsville.craigslist.org/mcd/d/florence-2018-kawasaki-ninja-400-abs/7120146414.html
https://huntsville.craigslist.org/mcd/d/florence-2016-harley-davidson-flhtk/7120083888.html
https://huntsville.craigslist.org/mcy/d/gurley-spyder-rtlimited-se6-low-miles/7119962123.html
https://huntsville.craigslist.org/mcd/d/florence-2008-suzuki-c50/7119788111.html
https://huntsville.craigslist.org/mcd/d/fayetteville-bad-credit-no-credit-ok-we/7119786915.html
https://huntsville.craigslist.org/mcy/d/owens-cross-roads-2012-harley-davidson/7119782972.html
https://huntsville.craigslist.org/mcd/d/florence-2015-harley-davidson-xg500/7119678665.html
https://huntsville.craigslist.org/mcd/d/lebanon-2008-harley-davidson-flhrse/7119676279.html
https://huntsville.craigslist.org/mcd/d/lebanon-2008-harley-davidson-flhr-road/7119676196.html
https://huntsville.craigslist.org/mcd/d/lebanon-2010-yamaha-xvs650-custom/7

https://huntsville.craigslist.org/mcd/d/florence-2013-honda-crf450r/7111560460.html
https://huntsville.craigslist.org/mcy/d/trenton-2011-electra-glide-classic-low/7111519081.html
https://huntsville.craigslist.org/mcy/d/winchester-1982-honda-ft500-ascot/7111315887.html
https://huntsville.craigslist.org/mcd/d/florence-2006-harley-davidson-flhx/7111283929.html
https://huntsville.craigslist.org/mcy/d/huntsville-2015-bmw-1200-gs/7110676343.html
Passing https://huntsville.craigslist.org/mcd/d/athens-2020-harley-davidson-sport-glide/7110376064.html
https://huntsville.craigslist.org/mcy/d/fayetteville-2014-honda-ctx-low-miles/7110351379.html
https://huntsville.craigslist.org/mcd/d/florence-2009-harley-davidson-xl1200l/7110226645.html
https://huntsville.craigslist.org/mcd/d/florence-2011-ducati-diavel-amg/7110134376.html
Https://Huntsville.Craigslist.Org/ Page 1 of 2 pages scraped successfully!
Passing https://huntsville.craigslist.org/mcd/d/florence-2011-ducati-diavel-amg/7110134376.html
https

https://montgomery.craigslist.org/mcy/d/montgomery-2008-yamaha-fjr-1300a-abs/7110448848.html
https://montgomery.craigslist.org/mcy/d/prattville-harley-ultra-classic/7107501674.html
Https://Montgomery.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Montgomery.Craigslist.Org/ complete!
~120 rows collected.


https://tuscaloosa.craigslist.org/ Posts = 7
Https://Tuscaloosa.Craigslist.Org/ Pages = 1
https://tuscaloosa.craigslist.org/mcy/d/thomasville-honda-vtx-1300/7122180661.html
https://tuscaloosa.craigslist.org/mcd/d/fayetteville-bad-credit-no-credit-ok-we/7119769610.html
https://tuscaloosa.craigslist.org/mcy/d/northport-1981-yamaha-xs650/7111909410.html
https://tuscaloosa.craigslist.org/mcy/d/tuscaloosa-2010-harley-davidson-ultra/7109082712.html
https://tuscaloosa.craigslist.org/mcy/d/tuscaloosa-2005-honda-shadow/7108986923.html
https://tuscaloosa.craigslist.org/mcy/d/coaling-victory-cross-country/7108393798.html
https://tuscaloosa.craigslist.org/mcy/d/fayette-harley-d

...

https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2014-bmw-1200-gs-in-gray/7117354951.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2014-yamaha-stratoliner/7117354957.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2018-bmw-1200-gs-lupin/7117354943.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2017-triumph-bonneville/7117354937.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2013-honda-cb1100-abs-in/7117354912.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2015-ktm-1290-super-duke/7117354846.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2019-harley-davidson-flhtcu/7117354654.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2014-bmw-800-gt/7117354607.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2015-harley-davidson-fltruse-cvo/7117354608.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2012-can-am-spyder-roadster-rs/7117354596.html
https://phoenix.craigslist.org/nph/mcd/d/las-vegas-2019-triumph-

https://phoenix.craigslist.org/evl/mcd/d/chandler-2019-bashan-storm-dual-sport/7116626584.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-2011-suzuki-boulevard-c50t-low/7116625988.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-2014-harley-davidson-fsb/7116625685.html
https://phoenix.craigslist.org/evl/mcd/d/2013-harley-davidson-heritage-softail/7116625131.html
https://phoenix.craigslist.org/nph/mcy/d/phoenix-2019-beta-430-rr/7116622102.html
https://phoenix.craigslist.org/evl/mcd/d/tempe-2016-harley-davidson-cvo-breakout/7116616783.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2018-kawasaki-br125jj-z125-pro-125/7116612559.html
https://phoenix.craigslist.org/nph/mcy/d/phoenix-2005-harley-davidson/7116606454.html
https://phoenix.craigslist.org/wvl/mcy/d/el-mirage-2005-dyna-lowrider/7116603587.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2018-harley-davidson-road-king-we/7116598759.html
https://phoenix.craigslist.org/cph/mcd/d/san-diego-2008-yamaha-yzfr6/

https://phoenix.craigslist.org/evl/mcd/d/bapchule-2015-kawasaki-vulcan-1700/7115940004.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2007-harley-davidson-flht/7115938607.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2006-honda-gl1800hpna/7115937329.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2019-harley-davidson-flhxs-street/7115937227.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2014-harley-davidson-flhr-road/7115936666.html
https://phoenix.craigslist.org/nph/mcd/d/phoenix-ride-today-low-payments-low/7115935965.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2008-harley-davidson-flhtcu/7115935923.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2015-suzuki-hayabusa-gsx1300-gsx/7115935833.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2018-kawasaki-kx450f/7115934506.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2015-harley-davidson-xl883n/7115933164.html
https://phoenix.craigslist.org/evl/mcd/d/bapchule-2010-harley-davidson-flstc

https://phoenix.craigslist.org/evl/mcd/d/mesa-2007-ducati-multistrada-1100-sport/7115396542.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2013-harley-davidson-flstn-softail/7115394972.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2013-harley-davidson-flstn-softail/7115293775.html
https://phoenix.craigslist.org/evl/mcd/d/higley-2003-kawasaki-vulcan-1500-mean/7115394345.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2016-harley-davidson-flhxse-cvo/7115394007.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2009-kawasaki-zg1400b9f-concours/7115392904.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2007-harley-davidson-softail/7115386086.html
https://phoenix.craigslist.org/cph/mcy/d/veradale-r1200-bmw-r1200s-sport-tour/7115366656.html
https://phoenix.craigslist.org/wvl/mcy/d/glendale-2012-kawasaki-concours-1400/7115328775.html
https://phoenix.craigslist.org/wvl/mcd/d/phoenix-2019-kawasaki-w800-cafe/7115275687.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2019-harl

https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-yamaha-tracer-900-in/7114640841.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2012-harley-davidson-flhx-street/7114640594.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2017-yamaha-scr950/7114640559.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-2017-ducati-multistrada-1200/7114640490.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-road/7114632971.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-street/7114632331.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-street/7114632308.html
https://phoenix.craigslist.org/evl/mcy/d/mesa-harley-davidson/7114590551.html
https://phoenix.craigslist.org/cph/mcy/d/el-mirage-2018-ktm-exc-500/7114586599.html
https://phoenix.craigslist.org/evl/mcy/d/san-tan-valley-harley-davidson/7114574990.html
https://phoenix.craigslist.org/evl/mcy/d/gilbert-road-king/7114543153.html
https://phoenix.craigs